# =============================================================================
# STEP 3: CUSTOMER CLUSTERING WITH K-MEANS
# Apply unsupervised ML to discover natural customer groupings
# =============================================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set random seed for reproducibility
RANDOM_STATE = 42

## 1. LOAD RFM DATA

In [ ]:
rfm = pd.read_csv('../data/processed/rfm_data.csv')
print(f"Loaded {len(rfm)} customers with RFM metrics")

# Features for clustering (we'll use log-transformed versions for skewed data)
features = ['Recency', 'Frequency', 'Monetary']

# Check distributions
print("\nFeature distributions:")
print(rfm[features].describe())

## 2. FEATURE ENGINEERING FOR CLUSTERING

In [ ]:
# Log transform to handle skewness (especially Monetary and Frequency)
rfm['Recency_log'] = np.log1p(rfm['Recency'])
rfm['Frequency_log'] = np.log1p(rfm['Frequency'])
rfm['Monetary_log'] = np.log1p(rfm['Monetary'])

# Additional features that might improve clustering
rfm['AvgOrderValue_log'] = np.log1p(rfm['AvgOrderValue'])

# Select features for clustering
cluster_features = ['Recency_log', 'Frequency_log', 'Monetary_log']
X = rfm[cluster_features]

print(f"\nClustering features: {cluster_features}")
print(f"Feature matrix shape: {X.shape}")

## 3. STANDARDIZATION (CRITICAL for K-Means)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nScaled features mean: {X_scaled.mean(axis=0).round(3)}")
print(f"Scaled features std: {X_scaled.std(axis=0).round(3)}")

## 4. FIND OPTIMAL K (Elbow Method + Silhouette Score)

In [ ]:
print("\n" + "="*60)
print("FINDING OPTIMAL NUMBER OF CLUSTERS")
print("="*60)

inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    print(f"K={k}: Inertia={kmeans.inertia_:.0f}, Silhouette={silhouette_scores[-1]:.3f}")

# Plot Elbow and Silhouette
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
ax1.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
ax1.set_title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axvline(x=4, color='red', linestyle='--', alpha=0.5, label='Optimal k=4')
ax1.legend()

# Silhouette plot
ax2.plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score by Number of Clusters', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axvline(x=4, color='red', linestyle='--', alpha=0.5, label='Optimal k=4')
ax2.legend()

plt.tight_layout()
plt.savefig('../reports/cluster_optimization.png', dpi=300, bbox_inches='tight')
plt.show()

# Choose optimal k (usually 4 or 5 for RFM)
optimal_k = 4  # Based on elbow and business interpretability
print(f"\n✅ Selected K={optimal_k} clusters")

# -----------------------------------------------------------------------------
# 5. APPLY K-MEANS CLUSTERING
# -----------------------------------------------------------------------------

kmeans = KMeans(n_clusters=optimal_k, random_state=RANDOM_STATE, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(X_scaled)

# Add cluster centers (inverse transform to original scale)
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)
centers_df = pd.DataFrame(np.expm1(centers_original), columns=features)
centers_df['Cluster'] = range(optimal_k)
print("\nCluster Centers (Original Scale):")
print(centers_df)

## 6. CLUSTER ANALYSIS & INTERPRETATION

In [ ]:
print("\n" + "="*80)
print("CLUSTER ANALYSIS")
print("="*80)

cluster_summary = rfm.groupby('Cluster').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum'],
    'AvgOrderValue': 'mean'
}).round(2)

cluster_summary.columns = ['Count', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary', 'Total_Revenue', 'Avg_Order_Value']
cluster_summary['Percentage'] = (cluster_summary['Count'] / len(rfm) * 100).round(1)
cluster_summary['Revenue_Share'] = (cluster_summary['Total_Revenue'] / cluster_summary['Total_Revenue'].sum() * 100).round(1)

# Sort by monetary value
cluster_summary = cluster_summary.sort_values('Avg_Monetary', ascending=False)
print(cluster_summary)

## 7. NAME CLUSTERS BASED ON CHARACTERISTICS

In [ ]:
def name_cluster(cluster_id):
    """Assign meaningful names based on cluster characteristics"""
    row = cluster_summary.loc[cluster_id]
    recency = row['Avg_Recency']
    frequency = row['Avg_Frequency']
    monetary = row['Avg_Monetary']
    
    if monetary > 3000 and recency < 50:
        return 'VIP Champions'
    elif monetary > 1000 and frequency > 5:
        return 'Loyal High-Value'
    elif recency < 100 and frequency < 3:
        return 'New Potentials'
    elif recency > 200:
        return 'At Risk / Churned'
    else:
        return 'Regular Customers'

# Apply names
cluster_names = {i: name_cluster(i) for i in range(optimal_k)}
rfm['Cluster_Name'] = rfm['Cluster'].map(cluster_names)
cluster_summary['Name'] = cluster_summary.index.map(cluster_names)

print("\n" + "="*80)
print("FINAL CLUSTER PROFILES")
print("="*80)
print(cluster_summary[['Name', 'Count', 'Percentage', 'Revenue_Share', 'Avg_Monetary']])

## 8. VISUALIZATION: 3D CLUSTER PLOT (Using PCA)

In [ ]:
# PCA for 2D visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

rfm['PCA1'] = X_pca[:, 0]
rfm['PCA2'] = X_pca[:, 1]

# Create cluster visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. PCA Scatter Plot
ax1 = axes[0, 0]
scatter = ax1.scatter(rfm['PCA1'], rfm['PCA2'], c=rfm['Cluster'], cmap='viridis', alpha=0.6, s=50)
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=11)
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=11)
ax1.set_title('Customer Clusters (PCA Projection)', fontsize=13, fontweight='bold')
legend1 = ax1.legend(*scatter.legend_elements(), title="Clusters", loc='upper right')
ax1.add_artist(legend1)

# 2. Recency vs Monetary by Cluster
ax2 = axes[0, 1]
for cluster in sorted(rfm['Cluster'].unique()):
    cluster_data = rfm[rfm['Cluster'] == cluster]
    ax2.scatter(cluster_data['Recency'], cluster_data['Monetary'], 
               label=f'Cluster {cluster}', alpha=0.6, s=50)
ax2.set_xlabel('Recency (Days)', fontsize=11)
ax2.set_ylabel('Monetary Value ($)', fontsize=11)
ax2.set_title('Recency vs Monetary by Cluster', fontsize=13, fontweight='bold')
ax2.legend()
ax2.set_yscale('log')

# 3. Frequency vs Monetary by Cluster
ax3 = axes[1, 0]
for cluster in sorted(rfm['Cluster'].unique()):
    cluster_data = rfm[rfm['Cluster'] == cluster]
    ax3.scatter(cluster_data['Frequency'], cluster_data['Monetary'], 
               label=f'Cluster {cluster}', alpha=0.6, s=50)
ax3.set_xlabel('Frequency (Orders)', fontsize=11)
ax3.set_ylabel('Monetary Value ($)', fontsize=11)
ax3.set_title('Frequency vs Monetary by Cluster', fontsize=13, fontweight='bold')
ax3.legend()
ax3.set_yscale('log')
ax3.set_xscale('log')

# 4. Cluster Comparison Heatmap
ax4 = axes[1, 1]
heatmap_data = cluster_summary[['Avg_Recency', 'Avg_Frequency', 'Avg_Monetary']].copy()
# Normalize for heatmap (0-1 scale)
heatmap_data_norm = (heatmap_data - heatmap_data.min()) / (heatmap_data.max() - heatmap_data.min())
sns.heatmap(heatmap_data_norm.T, annot=heatmap_data.T, fmt='.0f', cmap='YlOrRd', ax=ax4, cbar_kws={'label': 'Normalized Score'})
ax4.set_title('Cluster Characteristics Heatmap', fontsize=13, fontweight='bold')
ax4.set_xlabel('Cluster ID', fontsize=11)

plt.tight_layout()
plt.savefig('../reports/customer_clusters.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. COMPARE: MANUAL RFM SEGMENTS vs ML CLUSTERS

In [ ]:
print("\n" + "="*80)
print("COMPARISON: MANUAL SEGMENTS vs ML CLUSTERS")
print("="*80)

comparison = pd.crosstab(rfm['Segment'], rfm['Cluster_Name'], margins=True)
print(comparison)

## 10. SAVE RESULTS

In [ ]:
rfm.to_csv('../data/processed/customer_clusters.csv', index=False)
cluster_summary.to_csv('../data/processed/cluster_summary.csv')

print("\n✅ Clustering complete! Files saved:")
print("   - ../data/processed/customer_clusters.csv")
print("   - ../data/processed/cluster_summary.csv")

## 11. BUSINESS RECOMMENDATIONS BY CLUSTER

In [ ]:
print("\n" + "="*80)
print("BUSINESS RECOMMENDATIONS BY CLUSTER")
print("="*80)

recommendations = {
    'VIP Champions': {
        'characteristics': 'High value, recent purchases, frequent buyers',
        'action': 'VIP loyalty program, early access to new products, personalized service',
        'priority': 'HIGH - Protect these customers at all costs'
    },
    'Loyal High-Value': {
        'characteristics': 'Consistent high spenders with good frequency',
        'action': 'Upsell premium products, referral incentives, exclusive offers',
        'priority': 'HIGH - Grow their lifetime value'
    },
    'New Potentials': {
        'characteristics': 'Recent customers but low frequency/spend',
        'action': 'Onboarding campaigns, second-purchase incentives, education content',
        'priority': 'MEDIUM - Convert to loyal customers'
    },
    'At Risk / Churned': {
        'characteristics': 'Haven\'t purchased recently',
        'action': 'Win-back campaigns, special discounts, survey for feedback',
        'priority': 'MEDIUM - Reactivate before lost completely'
    }
}

for cluster_name, rec in recommendations.items():
    if cluster_name in rfm['Cluster_Name'].values:
        count = len(rfm[rfm['Cluster_Name'] == cluster_name])
        revenue = rfm[rfm['Cluster_Name'] == cluster_name]['Monetary'].sum()
        print(f"\n🔹 {cluster_name} ({count} customers, ${revenue:,.0f} revenue)")
        print(f"   Characteristics: {rec['characteristics']}")
        print(f"   Action: {rec['action']}")
        print(f"   Priority: {rec['priority']}")